1. Write a python program which searches all the product under a particular product from www.amazon.in. The 
product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for 
guitars. 

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
from selenium.webdriver.common.by import By
 # Set up the Chrome driver
driver = webdriver.Chrome()
driver.get('https://www.amazon.in/')
inputU = input('please enter product here--->')
search_bar = driver.find_element(By.XPATH,'//input[@id="twotabsearchtextbox"]')    # Finding the search bar using it's xpath
search_bar.send_keys(inputU)       # Inputing keyword to search 
search_button = driver.find_element(By.XPATH,'//input[@id="nav-search-submit-button"]')    # Finding the xpath of search button
search_button.click()        # Clicking the search button


please enter product here--->guitars


2. In the above question, now scrape the following details of each product listed in first 3 pages of your search 
results and save it in a data frame and csv. In case if any product has less than 3 pages in search results then 
scrape all the products available under that product name. Details to be scraped are: "Brand 
Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and 
“Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [2]:
import time
driver = webdriver.Chrome()
driver.get('https://www.amazon.in/')
inputU = input('please enter product here--->')
search_bar = driver.find_element(By.XPATH, '//input[@id="twotabsearchtextbox"]')    # Finding the search bar using its xpath
search_bar.send_keys(inputU)       # Inputting keyword to search 
search_button = driver.find_element(By.XPATH, '//input[@id="nav-search-submit-button"]')    # Finding the xpath of the search button
search_button.click()        # Clicking the search button

Brands = []
Prices = []
Expected_Delivery = []
Product_links = []

start = 0
end = 3  # Increase the number of pages you want to scrape

for page in range(start, end):
    # Scraping brands
    br = driver.find_elements(By.XPATH, '//span[@class="a-size-base-plus a-color-base a-text-normal"]')
    for i in br:
        if i.text is None :
            Brands.append("--")
        else:
            Brands.append(i.text)
    
    # Scraping prices
    pri = driver.find_elements(By.XPATH, '//span[@class="a-price"]')
    for i in pri:
        if i.text is None :
            Prices.append("--")
        else:
            Prices.append(i.text)
    
    # Scraping expected delivery
    exp_del = driver.find_elements(By.XPATH, '//div[@class="a-row s-align-children-center"]')
    for i in exp_del:
        if i.text is None :
            Expected_Delivery.append("--")
        else:
            Expected_Delivery.append(i.text)
    
    # Scraping product links
    product_links = driver.find_elements(By.XPATH, '//h2[@class="a-size-mini a-spacing-none a-color-base s-line-clamp-4"]/a')
    for link in product_links:
        product_url = link.get_attribute("href")
        if product_url is None or product_url.strip() == "":
            Product_links.append("--")
        else:
            Product_links.append(product_url)
        
    # Clicking next page button
    next_button = driver.find_element(By.XPATH, '//a[@class="s-pagination-item s-pagination-button"]')
    next_button.click()    
    time.sleep(5)

# Ensure all lists have the same length
min_length = min(len(Brands), len(Prices), len(Expected_Delivery), len(Product_links))
Brands = Brands[:min_length]
Prices = Prices[:min_length]
Expected_Delivery = Expected_Delivery[:min_length]
Product_links = Product_links[:min_length]

# Creating DataFrame
data = {
    "Brand": Brands,
    "Price": Prices,
    "Expected Delivery": Expected_Delivery,
    "Product URL": Product_links
}
df = pd.DataFrame(data)

# Saving to CSV
df.to_csv("amazon_products.csv", index=False)

# Printing DataFrame
print(df)

# Close the browser
driver.quit()


please enter product here--->guitars
                                                 Brand    Price  \
0    Kadence A281BK Professional Acoustic Rosewood ...   ₹6,499   
1    Kadence A281 Professional Acoustic Rosewood gu...   ₹6,499   
2    Kadence rosewood Guitar Frontier Series, Elect...   ₹5,499   
3    Kadence Frontier guitar with Online Guitar lea...   ₹4,999   
4    Yamaha F280 Acoustic Rosewood Guitar (Natural,...   ₹7,400   
..                                                 ...      ...   
164        Yamaha FG 800 Folk Acoustic Guitar, Natural  ₹12,399   
165  Kadence Frontier Series 40 Incheses wood Semi ...  ₹17,999   
166       Vault ST1 Premium Electric Guitar - Sunburst  ₹20,499   
167  Kadence Guitar Slowhand Series Premium Electri...   ₹5,499   
168           Intern INT-38C Acoustic Guitar Kit (Red)   ₹8,599   

                   Expected Delivery  \
0    Get it by Saturday, 10 February   
1    Get it by Saturday, 10 February   
2     Get it by Tomorrow, 9 February  

3. Write a python program to access the search bar and search button on images.google.com and scrape 10 
images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’.

In [5]:
import time
import os
import urllib.request  # Import urllib.request for downloading images

def scrape_images(keyword, num_images=10):
    driver = webdriver.Chrome()  # Make sure you have ChromeDriver installed and its path set in system environment variables.
    
    try:
        # Open Google Images
        driver.get("https://www.google.com/imghp")

        # Find and input the search bar
        search_bar = driver.find_element("name", "q")
        search_bar.send_keys(keyword)
        search_bar.send_keys(Keys.RETURN)

        # Wait for the search results to load
        time.sleep(2)

        # Scroll down to load more images (adjust the range as needed)
        for _ in range(3):
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)

        # Find and scrape image URLs using an  XPath expression
        image_elements = driver.find_elements(By.XPATH, '//img[@class="rg_i Q4LuWd"]')
        image_urls = [element.get_attribute('src') for element in image_elements[:num_images]]

        # Download images to a folder
        download_path = os.path.join(os.getcwd(), "downloaded_images", keyword)
        os.makedirs(download_path, exist_ok=True)

        for i, url in enumerate(image_urls):
            image_path = os.path.join(download_path, f"{keyword}_{i + 1}.jpg")
            urllib.request.urlretrieve(url, image_path)

        print(f"{num_images} images for '{keyword}' downloaded successfully to '{download_path}'.")

    except Exception as e:
        print(f"Error: {e}")

    finally:
        driver.quit()

keywords = ['fruits', 'cars', 'Machine Learning', 'Guitar', 'Cakes']

for keyword in keywords:
    scrape_images(keyword)


10 images for 'fruits' downloaded successfully to 'C:\Users\HP\downloaded_images\fruits'.
10 images for 'cars' downloaded successfully to 'C:\Users\HP\downloaded_images\cars'.
10 images for 'Machine Learning' downloaded successfully to 'C:\Users\HP\downloaded_images\Machine Learning'.
10 images for 'Guitar' downloaded successfully to 'C:\Users\HP\downloaded_images\Guitar'.
10 images for 'Cakes' downloaded successfully to 'C:\Users\HP\downloaded_images\Cakes'.


4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com
and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand 
Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, 
“Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the 
details is missing then replace it by “- “. Save your results in a dataframe and CSV. 

In [9]:
def scrape_flipkart(search_query):
    # Set up the Chrome driver
    driver = webdriver.Chrome()
    # Navigate to Flipkart search results page
    url = f"https://www.flipkart.com/search?q={search_query}"
    driver.get(url)
    time.sleep(2)  # Allow time for the page to load

    # Initialize lists to store scraped data
    brand_names = []
    roms = []
    cameras = []
    display_sizes = []
    battery_capacities = []
    prices = []
    product_urls = []

    # Extract data from each product on the page
    products = driver.find_elements(By.XPATH,'//div[@class="_1AtVbE col-12-12"]')
    for product in products:
        try:
            brand_name = product.find_element(By.XPATH,'//div[@class="_4rR01T"]').text
        except:
            brand_name = '-'


        try:
            rom = product.find_element(By.XPATH,'.//li[contains(text(), "ROM")]').text
        except:
            rom = '-'

        try:
            primary_camera = product.find_element(By.XPATH,'.//li[contains(text(), "Camera")]').text
        except:
            primary_camera = '-'

      
        try:
            display_size = product.find_element(By.XPATH,'.//li[contains(text(), "Display")]').text
        except:
            display_size = '-'

        try:
            battery_capacity = product.find_element(By.XPATH,'.//li[contains(text(), "Battery")]').text
        except:
            battery_capacity = '-'

        try:
            price =  battery_capacity = product.find_element(By.XPATH,'//div[@class="_30jeq3 _1_WHN1"]').text
        except:
            price = '-'

        try:
            product_url = product.find_element(By.TAG_NAME, 'a').get_attribute('href')
        except:
            product_url = '-'

        # Append the extracted data to the lists
        brand_names.append(brand_name)
        roms.append(rom)
        cameras.append(primary_camera)
        display_sizes.append(display_size)
        battery_capacities.append(battery_capacity)
        prices.append(price)
        product_urls.append(product_url)

    # Create a dataframe from the lists
    data = {
        'Brand Name': brand_names,
        'Rom': roms,
        'Camera': cameras,
        'Display Size': display_sizes,
        'Battery Capacity': battery_capacities,
        'Price': prices,
        'Product URL': product_urls
    }

    df = pd.DataFrame(data)
    print(df)

    # Save the dataframe to a CSV file
    df.to_csv('flipkart_smartphones.csv', index=False)

    # Close the browser
    driver.quit()

if __name__ == "__main__":
    search_query = input("Enter the smartphone you want to search for on Flipkart: ")
    scrape_flipkart(search_query)


Enter the smartphone you want to search for on Flipkart: Oneplus Nord, pixel 4A
                                    Brand Name                     Rom  \
0   OnePlus Nord CE 2 5G (Bahama Blue, 128 GB)                       -   
1   OnePlus Nord CE 2 5G (Bahama Blue, 128 GB)                       -   
2   OnePlus Nord CE 2 5G (Bahama Blue, 128 GB)   8 GB RAM | 128 GB ROM   
3   OnePlus Nord CE 2 5G (Bahama Blue, 128 GB)   8 GB RAM | 128 GB ROM   
4   OnePlus Nord CE 2 5G (Bahama Blue, 128 GB)    6 GB RAM | 64 GB ROM   
5   OnePlus Nord CE 2 5G (Bahama Blue, 128 GB)  12 GB RAM | 256 GB ROM   
6   OnePlus Nord CE 2 5G (Bahama Blue, 128 GB)   8 GB RAM | 128 GB ROM   
7   OnePlus Nord CE 2 5G (Bahama Blue, 128 GB)  12 GB RAM | 256 GB ROM   
8   OnePlus Nord CE 2 5G (Bahama Blue, 128 GB)    6 GB RAM | 64 GB ROM   
9   OnePlus Nord CE 2 5G (Bahama Blue, 128 GB)   8 GB RAM | 128 GB ROM   
10  OnePlus Nord CE 2 5G (Bahama Blue, 128 GB)   6 GB RAM | 128 GB ROM   
11  OnePlus Nord CE 2 5G (Bahama

5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [ ]:
pip install geopy


In [8]:
from geopy.geocoders import ArcGIS
def scrape_geospatial_coordinates (city):
    nom=ArcGIS()
    coordinates =nom.geocode(city)
    return coordinates
scrape_geospatial_coordinates('dharmavaram')

Location(Dharmavaram, Sri Sathyasai, Andhra Pradesh, (14.41328, 77.71049, 0.0))

6. Write a program to scrap all the available details of best gaming laptops from digit.in. 

In [10]:
# Set up the WebDriver
driver = webdriver.Chrome()

# Open the website
driver.get('https://www.digit.in/')

# Search for gaming laptops
search_bar = driver.find_element(By.XPATH,'//input[@id="woocommerce-product-search-field-0"]')
search_bar.send_keys('gaming laptops')
search_bar.submit()

Brands=[]
Ratings=[]
Price=[]


# Wait for the search results to load
time.sleep(2)

br=driver.find_elements(By.XPATH,'//h3[@class=" text-clamp text-clamp-2"]')
len(br)

for i in br:
    if i.text is None :
        
        Brands.append("--") 
    else:
        Brands.append(i.text)
#print(len(Investment_Type),Investment_Type)
                
print(Brands)


rat=driver.find_elements(By.XPATH,'//div[@class="rh_woo_star"]')
for i in rat:
    if i.text is None :
         Ratings.append("--") 
    else:
        Ratings.append(i.text)
        
print(Ratings)


pri=driver.find_elements(By.XPATH,'//span[@class="price"]')
for i in pri:
    if i.text is None :
         Price.append("--") 
    else:
        Price.append(i.text)
print(Price)





['Lenovo IdeaPad Gaming 3 15IAH7', 'Lenovo IdeaPad Gaming 3i', 'Alienware x14 Gaming Laptop', 'MSI Gaming Raider', 'Lenovo Legion 7', 'Dell 15 (2021) i5-10200H']
['★★★★★', '★★★★★', '★★★★★']
['₹79,831.00', '₹174,990.00', '₹328,800.00']


7. Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: 
“Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”. 

In [6]:
# Set up the WebDriver
driver = webdriver.Chrome()

# Define the URL you want to scrape
url = "https://www.forbes.com/billionaires/"

driver.get(url)

# Find the table element using its XPath
table_xpath = "//div[@class='Table_tableWrapper__fGr5G']"
table = driver.find_element(By.XPATH, table_xpath)

# Find all rows of the table using their XPath

cell_xpath = '//div[@class="TableRow_cell__db-hv Table_cell__houv9"]'

# Find the table element using its XPath
table = driver.find_element(By.XPATH, table_xpath)

# Find all cells containing required information within the table
cells = table.find_elements(By.XPATH, cell_xpath)

data = []

data = []

# Iterate through cells and extract data for "Rank", "Name", "Net worth", "Age", and "Source"
data = []

# Iterate through cells and extract data for "Rank", "Name", "Net worth", "Age", and "Source"
data = []

# Iterate through cells and extract data for "Rank", "Name", "Net worth", "Age", "Citizenship", "Source", and "Industry"
for i in range(0, len(cells), 7):
    # Check if there are enough cells remaining
    if len(cells) < i + 7:
        break
    
    # Extract text from specific cells (Rank, Name, Net worth, Age, Citizenship, Source, Industry)
    rank = cells[i].text
    name = cells[i + 1].text
    net_worth = cells[i + 2].text
    age = cells[i + 3].text
    citizenship = cells[i + 4].text
    source = cells[i + 5].text
    industry = cells[i + 6].text
    
    # Append data to the list
    data.append({
        "Rank": rank,
        "Name": name,
        "Net worth": net_worth,
        "Age": age,
        "Citizenship": citizenship,
        "Source": source,
        "Industry": industry
    })


    
    

    
    
   
# Convert the list of dictionaries to a pandas DataFrame
df = pd.DataFrame(data)
df
# Print the DataFrame
print(df)
    

# Close the webdriver
driver.quit()



    

    Rank                      Name Net worth Age    Citizenship  \
0      1  Bernard Arnault & family    $211 B  74         France   
1      2                 Elon Musk    $180 B  51  United States   
2      3                Jeff Bezos    $114 B  59  United States   
3      4             Larry Ellison    $107 B  78  United States   
4      5            Warren Buffett    $106 B  92  United States   
..   ...                       ...       ...  ..            ...   
195  195               Jin Baofang    $9.6 B  70          China   
196  195        Luo Liguo & family    $9.6 B  67          China   
197  195              Marijke Mars    $9.6 B  58  United States   
198  195               Pamela Mars    $9.6 B  62  United States   
199  195              Valerie Mars    $9.6 B  64  United States   

                 Source Industry  
0                  LVMH           
1         Tesla, SpaceX           
2                Amazon           
3                Oracle           
4    Berkshire Hatha

8. Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted 
from any YouTube Video. 

In [1]:
# Set up the WebDriver
driver = webdriver.Chrome()  # Replace with the path to your WebDriver executable

# Open the YouTube video
video_url = 'https://www.youtube.com/watch?v=1Xbv6JC_Zr4'  # Replace with the URL of the YouTube video
driver.get(video_url)

# Scroll to load comments
scroll_pause_time = 10  # Adjust the pause time as needed
scrolls = 1  # Adjust the number of scrolls as needed

for _ in range(scrolls):
    
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    time.sleep(scroll_pause_time)

# Extract comments, upvotes, and time
Comments=[]
comments = driver.find_elements(By.XPATH,'//div[@class="style-scope ytd-expander"]')
for i in comments:
    Comments.append(i.text)
print(Comments)


Upvotes=[]
upvotes = driver.find_elements(By.XPATH,'//span[@id="vote-count-middle"]')
scroll_pause_time = 10  # Adjust the pause time as needed
scrolls = 1  # Adjust the number of scrolls as needed
for i in upvotes:
    Upvotes.append(i.text)
print(Upvotes)


Times=[]
times = driver.find_elements(By.XPATH,'//a[@class="yt-simple-endpoint style-scope yt-formatted-string"]')
for i in times:
    Times.append(i.text)
print(Times)
# Close the WebDriver
driver.quit()




['Beautiful  always in my ,', 'Love this song ', 'beautiful', '', 'Botal Ka pata chala', 'Wow wow wow ', '', '', 'Love।song।', 'Kya mast song hai nice  ', 'So sweet songs', '', '', '', '', 'Hi all fb', 'How are fb', 'Hi', 'hello', 'Kon kon Instagram per sun kar Aya hai', '', ' I love this song ', 'X', '', 'E', 'Ye mtrcd addd Wale Santi se gana sunne Nehi deraha', '', '', '']
['7', '10', '4', '12', '15', '', '11', '5', '', '2', '31', '1', '1', '1', '2', '', '1', '5', '', '', '1', '1', '', '5', '1', '', '2', '']
['Ocean of Vibes', '', '', '', '7 days ago', '4 days ago', '12 days ago', '11 days ago', '6 days ago', '2 days ago', '11 days ago', '12 days ago', '22 hours ago', '4 days ago', '7 days ago', '2 days ago', '7 days ago', '6 days ago', '1 day ago', '3 days ago', '3 days ago', '13 days ago', '5 days ago', '1 day ago', '5 days ago', '7 days ago', '4 days ago', '2 weeks ago', '8 days ago', '18 hours ago', '4 days ago', '4 days ago']


9. Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in 
“London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall 
reviews, privates from price, dorms from price, facilities and property description.

In [1]:
# Set up the WebDriver
driver = webdriver.Chrome()
# Define the URL you want to scrape
url = "https://www.hostelworld.com/pwa/wds/s?q=London,%20England&country=London&city=London&type=city&id=3&from=2024-02-09&to=2024-02-12&guests=2&page=1"

# Open the URL
driver.get(url)

# Wait for the page to load
time.sleep(5) 
Hostel_info = []
hostel_name = driver.find_elements(By.XPATH, '//div[@class="property-name"]/span')
for i in hostel_name:
    Hostel_info.append(i.text)
    
print(Hostel_info)

# Get distance from city centre
Distance=[]
distance = driver.find_elements(By.XPATH, '//span[@class="distance-description"]')
for i in distance:
    Distance.append(i.text)
    
print(Distance)

 # Get ratings
Ratings=[]
ratings = driver.find_elements(By.XPATH, '//span[@class="number"]')
for i in ratings:
    Ratings.append(i.text)
print(Ratings)

 # Get total reviews
Total_reviews=[]
total_reviews = driver.find_elements(By.XPATH, '//div[@class="review"]')
for i in total_reviews:
    Total_reviews.append(i.text)
print(Total_reviews)


# Get privates from price
privates_price_dorms_price = driver.find_elements(By.XPATH,'//div[@class="property-accommodation-prices"]/div')
Privates_price_dorms_price_cleaned = []

for i in privates_price_dorms_price:
    price_text = i.text.replace('\n', ' ')  # Remove '\n' characters
    Privates_price_dorms_price_cleaned.append(price_text)

print(Privates_price_dorms_price_cleaned)


# Close the webdriver
driver.quit()

["Wombat's City Hostel London", 'Palmers Lodge - Swiss Cottage', "St Christopher's Village", 'NX London Hostel', 'Urbany Hostel London', 'Onefam Notting Hill', 'Generator London', 'Onefam Waterloo', 'Astor Hyde Park', 'Safestay London Elephant & Castle', 'Astor Museum Inn', 'No.8 Seven Sisters', 'No.8 Willesden Hostel London', 'Clink261', 'Smart Russell Square Hostel', 'Safestay London Kensington Holland Park', 'YHA London Oxford Street', 'Smart Hyde Park Inn Hostel', 'Destinations Hostels @ The Gallery', 'Queen Elizabeth Chelsea', 'Hostelle - women only hostel London', 'Prime Backpackers Angel', 'Kabannas London St Pancras', 'London Backpackers', 'Smart Camden Inn Hostel', 'Astor Kensington', "St Christopher's Hammersmith", 'Astor Victoria', 'Phoenix Hostel', 'Barmy Badger Backpackers']
['- 3.6km from city centre', '- 6.5km from city centre', '- 1.8km from city centre', '- 6.1km from city centre', '- 5.4km from city centre', '- 5.5km from city centre', '- 3km from city centre', '- 0.7